В этом примере мы покажем, как с помощью API Яндекс.Геокодера можно преобразовать адреса в геокоординаты, с которым можно работать в [Datalens](https://datalens.yandex.ru)

## Установка библиотек

 * requests для API Геокодера
 * clickhouse-driver для Clickhouse

In [1]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install clickhouse-driver

     |████████████████████████████████| 185 kB 1.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


## Получение ключа для API Геокодера

В переменную API_KEY нужно записать свой ключ для API Геокодера.
Получить ключ можно по [ссылке](https://yandex.ru/dev/maps/geocoder/)

In [1]:

API_KEY = ''

## Работа с API Геокодера

Напишем небольшой класс для работы с Геокодером

In [1]:
import requests
from dataclasses import dataclass

@dataclass
class YandexGeocoder:
    api_key: str
    geocoder_url: str = 'https://geocode-maps.yandex.ru/1.x'

    def adress_to_geopoint(self, address: str) -> str:
        """
        Преобразование адреса в геокоординаты в формате Datalens
        """
        response = requests.get(self.geocoder_url, params={
            'apikey': self.api_key,
            'geocode': address,
            'format': 'json',
        })
        response.raise_for_status()

        result = response.json()['response']['GeoObjectCollection']['featureMember']
        if not result:
            return None

        lat, lon = result[0]['GeoObject']['Point']['pos'].split(' ')
        return self._to_datalens_format(lon, lat)
    
    def _to_datalens_format(self, lon, lat):
        return f'[{lon},{lat}]'

## Получение данных

Будем работать с данными из демонстрационного Clickhouse [Datalens](https://datalens.yandex.ru).

На первом шаге подготовим клиент Clickhouse

In [2]:
from clickhouse_driver import Client as CHClient

ch_client = CHClient(
    'rc1a-ckg8nrosr2lim5iz.mdb.yandexcloud.net',
    user='samples_ro',
    password='MsgfcjEhJk',
    database='samples',
    port=9440,
    secure=True,
)

Затем выгрузим данные из таблицы в переменную ch_data

In [13]:
import pandas as pd
all_files = ["rus_cityes.csv"]
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df

,Russia,moscow
0,Russia,rostov-on-don


In [40]:
ch_data2 = ch_data[0:1000]

In [41]:
 
for i in range(len(ch_data)):
    ch_data[i] = "Russia, " + ch_data[i]
ch_data

['Russia, Kulunda',
 'Russia, Spitsevka',
 'Russia, Pokanaevka',
 'Russia, Belorus',
 'Russia, Pudost',
 'Russia, Nema',
 'Russia, Voronezhskoe Malanino',
 'Russia, Rzhevskoe',
 'Russia, Nekrasovskoe',
 'Russia, Vakhrushi (pgt)',
 'Russia, Slyudyanka',
 'Russia, Duminichi',
 'Russia, Sződ',
 'Russia, Omutinskoe',
 'Russia, Berezovo',
 'Russia, Blagoveschensk',
 'Russia, Zhilgorodok',
 'Russia, Obvinsk',
 'Russia, Sosnovy',
 'Russia, Fedorovka (olkhovskoe S/p)',
 'Russia, Terengul (selo)',
 'Russia, Vorkuta',
 'Russia, Nizhnyaya Ternovka',
 'Russia, Cherdyn',
 'Russia, Zakharov',
 'Russia, Yutaza',
 'Russia, Nizhneangarsk',
 'Russia, Goryachy Klyuch',
 'Russia, Lepley',
 'Russia, Romodanovo',
 'Russia, Lyskovo',
 'Russia, Moshenskoe',
 'Russia, Shinsha',
 'Russia, Bolshoy',
 'Russia, Galenki',
 'Russia, Grunin Vorgol (selo)',
 'Russia, Ostrovskoe (poselok)',
 'Russia, Brusentsevo',
 'Russia, Valday',
 'Russia, Makushino',
 'Russia, Slavyansk-Na-Kubani',
 'Russia, Sandovo',
 'Russia, Che

## Геокодирование

Преобразуем адреса магазинов из колонки ShopAddress в геокоординаты

In [11]:
geocoder = YandexGeocoder(api_key=API_KEY)

In [42]:
encoded_data = []
for adress in ch_data:
    s = geocoder.adress_to_geopoint(adress)
    print(adress, s)
    encoded_data.append(s)
# encoded_data

Russia, Kulunda [52.563758,78.946507]
Russia, Spitsevka [45.120904,42.507013]
Russia, Pokanaevka [56.514967,97.659915]
Russia, Belorus [55.214671,34.184607]
Russia, Pudost [59.616298,30.041693]
Russia, Nema [57.505782,50.498479]
Russia, Voronezhskoe Malanino [52.104453,39.329168]
Russia, Rzhevskoe [55.081823,21.757394]
Russia, Nekrasovskoe [57.674363,40.369004]
Russia, Vakhrushi (pgt) [58.684661,50.026415]
Russia, Slyudyanka [51.656501,103.718845]
Russia, Duminichi [53.933216,35.105649]
Russia, Sződ [61.698657,99.505405]
Russia, Omutinskoe [56.475767,67.664674]
Russia, Berezovo [54.045788,36.413821]
Russia, Blagoveschensk [50.257456,127.534611]
Russia, Zhilgorodok [51.997463,55.507378]
Russia, Obvinsk [58.495365,54.850628]
Russia, Sosnovy [59.904225,29.09221]
Russia, Fedorovka (olkhovskoe S/p) [51.870924,58.041839]
Russia, Terengul (selo) [53.827183,77.160908]
Russia, Vorkuta [67.493504,64.050113]
Russia, Nizhnyaya Ternovka [45.490682,42.326218]
Russia, Cherdyn [60.401192,56.482166]
Ru

In [43]:
len(encoded_data)

1000

In [44]:
import pandas as pd

In [61]:
ch_data2 = [w[8:] for w in ch_data]

In [62]:
df = pd.DataFrame(encoded_data, ch_data2[:1000])

In [63]:
df.to_csv("coord_city5.csv")